In [ ]:
from datasets import load_dataset, get_dataset_split_names

get_dataset_split_names("curaihealth/medical_questions_pairs")

In [ ]:
get_dataset_split_names("ruslanmv/ai-medical-chatbot")

In [ ]:
ds = load_dataset("curaihealth/medical_questions_pairs")

In [ ]:
ds = ds["train"].train_test_split(test_size=0.2, seed=42)

In [ ]:
ds["train"][0]

In [ ]:
ds["train"][:3]

In [ ]:
ds

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

In [ ]:
tokenizer(
    ds["train"][0]["question_1"], ds["train"][0]["question_2"], return_tensors="pt"
)

In [ ]:
def tokenization(example):
    return tokenizer(
        example["question_1"], example["question_2"], return_tensors="pt", padding=True
    )


tokenized_ds = ds.map(tokenization, batched=True)
print(tokenized_ds)

In [ ]:
tokenized_ds["train"].features

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2
)

In [ ]:
import wandb

wandb.init(project="medical_bot", name="bert_similarity")

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./model",
    # evaluation_strategy="steps",
    eval_steps=50,
    logging_steps=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    report_to="wandb",  # логирование в W&B
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    compute_metrics=lambda p: {
        "accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()
    },
)
trainer.train()